## Performance Analysis

In this notebook, we will compare the performance of various neural network models, focusing on hyperparameter tuning and optimization strategies.

### Step 1: Visualize and Save Model Architecture

To start, we will visualize and save the architecture of our neural network models using TensorBoard. This will help us understand the structure of our models and monitor their training progress.

Here’s how we will proceed:

1. **Import TensorBoard**: Use the `torch.utils.tensorboard` module to set up TensorBoard.
2. **Create a Summary Writer**: Initialize a TensorBoard `SummaryWriter` to log the model architecture.
3. **Visualize the Model**: Pass a dummy input through the model and use TensorBoard to visualize the architecture.
4. **Save the Visualization**: Save the visualization to disk for later review.


In [5]:
!wget https://raw.githubusercontent.com//hhosseinian/Face_Recognition/main/Models1.py # Uncomment this line for Running over Collab
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from Models1 import Net

# Create a dummy input tensor
dummy_input = torch.randn(1, 1, 244, 244)

# Initialize the model
model = Net()

# Set up TensorBoard SummaryWriter
writer = SummaryWriter()

# Visualize the model
writer.add_graph(model, dummy_input)

# Close the writer
writer.close()

print("Model architecture visualization saved to TensorBoard.")

--2024-08-06 04:43:01--  https://raw.githubusercontent.com//hhosseinian/Face_Recognition/main/Models1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /hhosseinian/Face_Recognition/main/Models1.py [following]
--2024-08-06 04:43:01--  https://raw.githubusercontent.com/hhosseinian/Face_Recognition/main/Models1.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3496 (3.4K) [text/plain]
Saving to: ‘Models1.py.2’

Models1.py.2        100%[===================>]   3.41K  --.-KB/s    in 0s      

2024-08-06 04:43:01 (42.6 MB/s) - ‘Models1.py.2’ saved [3496/3496]

mat1 and mat2 shapes cannot be multiplied (1x30976 and 25600x2000)
Error occurs, No graph saved


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30976 and 25600x2000)